In [1]:
import gradio as gr

from src.engine import GradioEngine
import src.fe_variables as app_metadata

# Gradio app configuration

## Tabbed interface with tabs:
    # -Select Logbook: Takes user input on .pdf file to parse, creates Canvas, and creates all Canvas pages.
    # -Ask Question: Takes user question as input, updates and executes the formula on the page with new question.
    # -Full Logbook Data: Shows all of the data extracted from the logbook
    # -FAQ: Answers Frequently Asked Questions

In [2]:
# Create GradioEngine Object
session = GradioEngine()
# Pull the most current list of filenames from the blob storage
session.list_blob()


# ====== #
# Tab1: SelectLogbook
# ====== #
with gr.Blocks(title=app_metadata.TAB1_PAGE_NAME, theme=gr.themes.Glass()) as SelectLogbook:
    # Instructions
    instructions = gr.Markdown(value=app_metadata.TAB1_INSTRUCTIONS)
    # Input: Filename
    filename = gr.Textbox(label="Filename: ", placeholder="Ex. Xerox Scan 12345.pdf")
    # Output: Confirmation message to user
    confirmation_msg = gr.Textbox(label="Confirmation Message: ", placeholder="")
    # Buttons
    tab1_btn = gr.Button("Run")
    # clear = gr.ClearButton([filename, confirmation_msg])
    examples = gr.Examples(examples=session.list_of_filenames_in_blob, inputs=[filename])
    # examples_btn = gr.Button("Load Examples")

    # Execute
    tab1_btn.click(session.tab1_engine, filename, confirmation_msg)


# ====== #
# Tab2: AskQuestion
# ====== #
TAB2_INSTRUCTIONS = "<h3>Instructions</h3>"\
                    f"<p>Ask a question about information in selected logbook </p> "#:<b>{session.full_filename}</b> </p>"

with gr.Blocks(title=app_metadata.TAB2_PAGE_NAME, theme=gr.themes.Glass()) as AskQuestion:
    # Instructions
    instructions = gr.Markdown(value=TAB2_INSTRUCTIONS)
    # Input
    tab2_question = gr.Textbox(label="Question:", placeholder="Ex. List the N2 Intervals.")
    # Buttons
    tab2_run_btn = gr.Button("Run")
    # Output
    tab2_response = gr.DataFrame(label="Answer:", interactive=True)
    # More Buttons & Export Data
    tab2_export_btn = gr.Button("Download Table as csv")
    tab2_csv = gr.File(interactive=False, visible=False)
    tab2_clear = gr.ClearButton([tab2_question, tab2_response, tab2_csv])

    # Execute
    tab2_run_btn.click(session.tab2_engine, tab2_question, tab2_response)
    tab2_export_btn.click(session.export_csv, tab2_response, tab2_csv)


# ====== #
# Tab3: Full Logbook Data
# ====== #
TAB3_INSTRUCTIONS = "<h3>Instructions</h3>"\
                     "<p>Click the run button to return the full table of data extracted from the logbook.</p>" \
                    #  f"<p>Selected Logbook: <b>{session.full_filename}</b></p>"

with gr.Blocks(title=app_metadata.TAB3_PAGE_NAME, theme=gr.themes.Glass()) as FullLogbook:
    # Instructions
    instructions = gr.Markdown(value=TAB3_INSTRUCTIONS)
    # Buttons
    tab3_btn = gr.Button("Run")
    # Output
    tab3_response = gr.DataFrame(label="Answer:", interactive=True)
    # More Buttons & Export Data
    tab3_export_btn = gr.Button("Download Table as csv")
    tab3_csv = gr.File(interactive=False, visible=False)
    tab3_clear = gr.ClearButton([tab3_btn, tab3_response, tab3_csv])

    # Execute
    tab3_btn.click(session.tab3_engine, tab3_btn, tab3_response)
    tab3_export_btn.click(session.export_csv, tab3_response, tab3_csv)

# ====== #
# Tab4: FAQ
# ====== #
with gr.Blocks(title=app_metadata.TAB4_PAGE_NAME, theme=gr.themes.Glass()) as FAQ:
    # Instructions
    instructions = gr.Markdown(value=app_metadata.TAB4_INSTRUCTIONS)


# ====== #
# Put it together
# ====== #
demo = gr.TabbedInterface([SelectLogbook, AskQuestion, FullLogbook, FAQ], [app_metadata.TAB1_PAGE_NAME, app_metadata.TAB2_PAGE_NAME, app_metadata.TAB3_PAGE_NAME, app_metadata.TAB4_PAGE_NAME], title=app_metadata.APP_NAME)

if __name__ == "__main__":
    demo.launch()

Environments available in config: ['AriesProd1', 'AriesTest']
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


['Xerox Scan_02152024100620.pdf', 'Xerox Scan_02152024140510.pdf']
{'id': '60176d2d-c27d-40f6-a87d-f61a95704edb', 'version': '6a5bdb08-d568-4bdf-9a31-a8e14e48caf4', 'applicationMetadata': None, 'settings': {'supportedDataSources': {'AzureBlob': [{'accountName': 'rcdatat101teststorage', 'id': 'rcdatat101teststorage', 'displayName': 'Research Canvas Data Storage', 'type': 'AzureBlob'}], 'AzureDataLake_Gen2': [{'accountName': 'chevrondatalake', 'id': 'Prod-2', 'displayName': 'Produced', 'type': 'AzureDataLake_Gen2'}, {'accountName': 'chevrondatalakedev', 'id': 'chevrondatalakedev', 'displayName': 'Development', 'type': 'AzureDataLake_Gen2'}], 'DedicatedSqlPool': [{'accountName': 'tcp:rcdatasynapseprod.sql.azuresynapse.net,1433', 'id': 'tcp:rcdatasynapseprod.sql.azuresynapse.net,1433', 'displayName': 'Synapse SQL pool', 'type': 'DedicatedSqlPool'}], 'AzureDataLake': [], 'AnalysisService': []}, 'serviceVersion': 'V2', 'flightSettings': {'uiJson': '{"lookup":true,"copyPaste":true,"cluster":t

# Testing

In [ ]:
# Create session object
session = GradioEngine()

# Testing tab1
session.tab1_engine(filename="Xerox Scan_02152024100620.pdf")

# Testing tab2 (have to run tab 1 first)
tab2_data = session.tab2_engine(user_question="List the N2 entries")
print(tab2_data)

# Testing tab3 (have to run tab 1 first)
tab3_data = session.tab3_engine(input="gimme")
print(tab3_data)